In [1]:
import numpy as np
import pygrib
from scipy import interpolate, optimize 

import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap, cm

import multiprocessing as mp

In [2]:
# linear spline functions

def linear_splines_unif(data, num_knots=10, level_width=1):   
    '''
    Calculates piecewise linear splines for quantile data using specified number of 
    knots uniformly spaced and returning interpolated approximation at every 
    quantile level with level_width.
    ''' 

    # calculating where cdf starts being nonzero (all zero cdf's should not be inputted)
    levels = np.array(range(1,100))
    knot_ = np.where(data > 0)[0].min() - 1   
    if knot_ > 1:
        knots = np.unique(np.linspace(knot_-1, 98, num_knots-1, dtype=int))
        knots = np.insert(knots, 0, 0)
    else:
        knots = np.unique(np.linspace(0, 98, num_knots, dtype=int))
        
    # approx = interpolate.interp1d(knots+1, data[knots], assume_sorted=True) 
    # return approx(levels[level_width-1::level_width])
    return np.interp(levels[level_width-1::level_width], knots+1, data[knots])

def linear_splines(x, num_knots, *params):
    '''
    Function to be used in scipy.optimize.curve_fit in linear_splines_var function.
    '''

    knot_vals = list(params[0][0:num_knots])
    knots = list(params[0][num_knots:])
    return np.interp(x, knots, knot_vals)

def linear_splines_var(data, num_knots, level_width):
    '''
    Calculates piecewise linear splines for quantile data using specified number of
    knots with optimized placement and returning interpolated approximation at every
    quantile level with level_width.
    '''
#     checking if cdf is all zero
#     if data[-1] == 0:
#         return np.zeros(int(np.floor(100/level_width)))

    # setting up intial value of parameters
    p_0 = np.linspace(0,98,num_knots).astype(int)
    p_0 = np.hstack([data[p_0], p_0])

    # try to fit parameters with RuntimeError exception that returns linear_splines_unif
    # that uses uniformly space knots
    try:
        fit, _ = optimize.curve_fit(lambda x, *params : linear_splines(x, num_knots, params), 
                np.linspace(1,99,99), data, p_0)
        levels = np.linspace(1,99,99)
        levels = levels[level_width-1::level_width]
        return np.interp(levels, fit[num_knots:], fit[:num_knots])
    except RuntimeError:
        return linear_splines_unif(data, num_knots=num_knots, level_width=level_width)

In [7]:
# read in grib file
fn_grb = 'data/blend20220324.t12z.qmd.f018.co.grib2'
# fn_grb = 'data/blend20220210.t12z.qmd.f018.co.grib2'
# fn_grb = 'data/blend.20220525t12z.qmd.f018.co.grib2'
ds_grb = pygrib.open(fn_grb)

# latitude and longitude grid values
lat, long = ds_grb.message(2).data()[1:]

# extracting data
precip_shape = lat.shape
precip_levels = np.zeros(shape=(99,)+precip_shape)
temp_levels = precip_levels
for i in range(99):
    precip_levels[i,:,:] = ds_grb.message(i+2).data()[0] # ACPC:surface:12-18 hour acc fcst
    temp_levels[i,:,:] = ds_grb.message(i+215).data()[0] # TMP:2 m above ground:0-18 hour max

In [ ]:
# checking for monotonicity
for i in range(lat.shape[0]):
    for j in range(lat.shape[1]):
        for level in range(99-1):
            if precip_levels[level,i,j] > precip_levels[level+1,i,j]:
                print(f'Issue with precip at {i}, {j}.')
            if temp_levels[level,i,j] > temp_levels[level+1,i,j]:
                print(f'Issue with temp at {i}, {j}.')

In [ ]:
# level_width = 30
# precip_levels_approx_unif = np.zeros(shape=(int(np.floor(100/level_width)),)+lat.shape)
# for i in range(precip_shape[0]):
#     for j in range(precip_shape[1]):
#         precip_levels_approx_unif[:,i,j] = linear_splines_unif(data=precip_levels[:,i,j], 
#                                     num_knots=10, level_width=level_width)      
# np.save('precip_levels_approx_unif', precip_levels_approx_unif)
precip_levels_approx_unif = np.load('precip_levels_approx_unif.npy')

In [ ]:
count = [0,0,0]
level_width = 30
global precip_levels_approx_var
precip_levels_approx_var = np.zeros(shape=(int(np.floor(100/level_width)),)+lat.shape)
# for i in range(precip_shape[0]):
#     for j in range(precip_shape[1]):
#         precip_levels_approx_var[:,i,j] = linear_splines_var(precip_levels[:,i,j], 10, level_width)
#         if sum(count) in [100000, 500000, 1000000, 1500000, 2000000, 2500000, 3000000, 3500000]:
#             print(sum(count),lat.shape[0]*lat.shape[1])
# np.save('precip_levels_approx_var', precip_levels_approx_var)
precip_levels_approx_var = np.load('precip_levels_approx_var.npy')

In [ ]:
precip_levels_approx_var = np.load('precip_levels_approx_var.npy')
precip_levels_approx_var_ = np.load('precip_levels_approx_var_.npy')

In [ ]:
# graphing using basemap

def Basemap_plot(data, long, lat, var_name='Precipitation', diff=False, levels=False):

    map = Basemap(llcrnrlon=-123.,llcrnrlat=20., 
                   urcrnrlon=-59., urcrnrlat=48., 
                   projection='lcc', 
                   lat_1=38.5,
                   lat_0=38.5,
                   lon_0=-97.5,
                   resolution='l')

    # draw coastlines, country boundaries, fill continents
    map.drawcoastlines(linewidth=0.25)
    map.drawcountries(linewidth=0.25)
    map.fillcontinents(color='xkcd:white',lake_color='xkcd:blue')



    # draw the edge of the map projection region (the projection limb)
    map.drawmapboundary(fill_color='xkcd:blue')
    map.drawstates()

    # draw lat/lon grid lines every 30 degrees.
    map.drawmeridians(np.arange(-180,180,30))
    map.drawparallels(np.arange(-90,90,30))

    x, y = map(long, lat)
    
    if diff:
        data_abs_max = int(np.ceil(max(np.abs(data.min()),np.abs(data.max()))))
        if not(isinstance(levels, list) or isinstance(levels, np.ndarray)):
            levels = list(range(-data_abs_max,data_abs_max+1))
        plt.pcolormesh(x, y, data,
                       norm=colors.Normalize(vmin=levels[0], vmax=levels[-1]),
                       cmap='seismic', shading='nearest')
        # map.contourf(x, y, data, 16, levels=levels, cmap='seismic')
        map.colorbar()
    else:
        map.contour(x, y, data, 16, linewidths=1.5)
    plt.title(var_name)
    plt.show()

In [ ]:
var_name = 'Precipitation at Probability Level 90%'
Basemap_plot(data=precip_levels[90-1,:,:], long=long, lat=lat, var_name=var_name)

In [ ]:
var_name = 'Difference at Probability Level 90%'
diff = precip_levels_approx_var_[2,:,:]-precip_levels[90-1,:,:]
Basemap_plot(data=diff, long=long, lat=lat, var_name=var_name, diff=True)

In [ ]:
precip_levels_flat = np.zeros(shape=(99,lat.shape[0]*lat.shape[1]))
for element in range(lat.shape[0]*lat.shape[1]):
    i = int(np.floor(element/lat.shape[1]))
    j = int(element % lat.shape[1])
    precip_levels_flat[:,element] = precip_levels[:,i,j]

In [ ]:
nonzero_idx = np.where(precip_levels_flat[-1,:] != 0)
nonzero_idx[0].shape[0]
lat.shape[1]

In [ ]:
nonzero_idx = np.where(precip_levels[-1,:,:] != 0)
n = 7906
# n = int(np.random.uniform(0,nonzero_idx[0].shape[0]-1))
i = nonzero_idx[0][n]
j = nonzero_idx[1][n]
data_max = precip_levels[:,i,j].max()
print(data_max)

In [ ]:
approx = linear_splines_var(precip_levels[:,i,j], 10, 1)
approx_ = linear_splines_var(precip_levels[:,i,j], 10, 30)

In [ ]:
levels = np.linspace(1,99,99)
plt.plot(precip_levels[:,i,j], levels, c='xkcd:black', label='orig')
plt.plot(precip_levels_approx_var_[:,i,j], [30,60,90], c='xkcd:red', label='old_code')
plt.plot(approx_, [30,60,90], c='xkcd:orange', label='new_same_level')
plt.plot(approx, levels, c='xkcd:blue', label='all_levels')
plt.legend()

In [ ]:
precip_levels_approx_var_.shape

In [ ]:
idx1 = np.where(precip_levels[-2,:,:] != 0)
for n in range(idx1[0].shape[0]):
    i = idx1[0][n]
    j = idx1[1][n]
    if precip_levels[-1,i,j] == 0:
        print(i,j)

In [ ]:
i=62
j=457
plt.plot(np.linspace(1,99,99),precip_levels[:,i,j])
plt.xlabel('prob level')
plt.ylabel('mm of precip')

In [ ]:
precip_levels[[96,97,98],62,457]

In [ ]:
idx2 = np.where(precip_idx1[-1,:,:] == 0)
precip_idx2 = precip_idx1[:,idx2[0],idx2[1]]

In [ ]:
precip_idx1.shape